# Decision Tree

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import generate_graph, generate_random_data
from collections import defaultdict
from typing import Optional, Self, List, Tuple
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
class GirlType(Enum):
    WIFEY_MATERIAL = "Wifey Material"
    PROFESSIONAL_PARTNER = "Professional Partner"
    SEXUALLY_ENTICING = "Sexually Enticing"
    STRANGER = "Stranger"

class BinaryTreeNode:
    def __init__(self, left_child: Optional[Self] = None, right_child: Optional[Self] = None) -> None:
        self.left = left_child
        self.right = right_child
        self.data = dict()

class GirlTypeDecisionTree:
    read_iteration = 0
    def __init__(self):
        self._label_count = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        self._gini_gains = defaultdict(float)
        self._root = BinaryTreeNode()
        pass
    
    @staticmethod
    def _weighted_gini_impurity(girl_type_count: defaultdict, N_parent: int) -> float:
        N_child = sum(girl_type_count.values())
        gini_impurity = 1 - sum([(count / N_child) ** 2 for count in girl_type_count.values()])
        return N_child/N_parent * gini_impurity
    
    def _set_label_count(self, X: np.ndarray, y: np.ndarray) -> None:
        for feature in range(len(X.T)):
            unique_feature_val = np.sort(np.unique(X.T[feature]))
            thresholds = set((unique_feature_val[:-1] + unique_feature_val[1:]) / 2)
            for threshold in thresholds:
                for girl_type in [c.value for c in GirlType]:
                    self._label_count[(feature, threshold)]["<="][girl_type] = sum((X.T[feature] <= threshold) & (y == girl_type))
                    self._label_count[(feature, threshold)][">"][girl_type] = sum((X.T[feature] > threshold) & (y == girl_type))
    
    def _get_best_split_property(self, y: np.ndarray) -> Tuple[float, Tuple[int, np.float64]]:
        N = len(y)
        for split_condition in self._label_count.keys():
            left_girl_type_count = self._label_count[split_condition]["<="]
            right_girl_type_count = self._label_count[split_condition][">"]
            weighted_left = self._weighted_gini_impurity(left_girl_type_count, N)
            weighted_right = self._weighted_gini_impurity(right_girl_type_count, N)
            self._gini_gains[split_condition] = sum([weighted_left, weighted_right])
        best_split = min(self._gini_gains, key=self._gini_gains.get)
        gini_impurity = self._gini_gains[best_split]
        return (best_split, gini_impurity)
    
    def _decide_prediction(self, y: np.ndarray, current_node: BinaryTreeNode) -> None:
        value_counts = dict(zip(*np.unique(y, return_counts=True)))
        predicted = max(value_counts, key=value_counts.get)
        probability = round(value_counts[predicted] / sum(value_counts.values()) * 100)
        current_node.data = {"prediction": predicted, "probability": str(probability) + "%"}

    def fit(self, 
            X: np.ndarray, y: np.ndarray, 
            current_node: Optional[BinaryTreeNode] = None, 
            N: Optional[int] = 0, parent_impurity: Optional[int] = 1
            ) -> None:
        self._label_count.clear()
        self._gini_gains.clear()
        if current_node is None: current_node = self._root
        self._set_label_count(X, y)
        if len(self._label_count.keys()) == 0: 
            self._decide_prediction(y, current_node)
            return
        (feature, threshold), child_impurity = self._get_best_split_property(y)
        if child_impurity > parent_impurity:
            self._decide_prediction(y, current_node)
            return
        current_node.data = {"feature": feature, "threshold": threshold}
        left_mark, right_mark = [X.T[feature] <= threshold, X.T[feature].T > threshold]
        current_node.left = BinaryTreeNode()
        self.fit(X[left_mark], y[left_mark], current_node=current_node.left, N=N+1, parent_impurity=child_impurity)
        current_node.right = BinaryTreeNode()
        self.fit(X[right_mark], y[right_mark], current_node=current_node.right, N=N+1, parent_impurity=child_impurity)
    
    def predict(self, X: np.ndarray) -> List:
        def get_prediction(Xi: np.ndarray, current_node: BinaryTreeNode) -> str:
            if "prediction" in current_node.data.keys(): return current_node.data["prediction"]
            if Xi[current_node.data['feature']] <= current_node.data['threshold']:
                return get_prediction(Xi, current_node.left)
            else:
                return get_prediction(Xi, current_node.right)
        y_predict = []
        for Xi in X:
            y_predict.append(get_prediction(Xi, self._root))
        return y_predict
    
    def read_tree(self, 
                current_node: Optional[BinaryTreeNode] = None, 
                parent_node: Optional[BinaryTreeNode] = None, 
                result: Optional[List] = [],
                sign: Optional[str] = None
                ) -> None:
        if parent_node is not None:
            if parent_node.data:
                feature, threshold = parent_node.data.values()
                result.append(f"(Feature {feature} {sign} {threshold})")
            if current_node.left is None: 
                GirlTypeDecisionTree.read_iteration += 1
                print(GirlTypeDecisionTree.read_iteration, end=". ")
                print(' -> '.join(result), end=" -> ")
                print(' | '.join(current_node.data.values()), end="\n")
                result.pop()
                return
            self.read_tree(current_node.left, current_node, result,"<=")
            self.read_tree(current_node.right, current_node, result,">")
        else:
            result = []
            GirlTypeDecisionTree.read_iteration = 0
            self.read_tree(self._root.left, self._root, result,"<=")
            self.read_tree(self._root.right, self._root, result,">")
        if len(result) > 0: result.pop()

In [39]:
df = generate_random_data.get_decision_tree_data(1250)
df["Wifey Material?"] = df["Girl Type"].map(lambda girl_type: 1 if girl_type == GirlType.WIFEY_MATERIAL.value else 0)
df = df.drop("Girl Type", axis=1)
df.sample(10)
new_df = df.copy()
new_df["Facial Attractiveness"] = new_df["Facial Attractiveness"].map({"Ugly": 0, "Mid": 1, "Beautiful": 2}).astype(int)
new_df["Body Attractiveness"] = new_df["Body Attractiveness"].map({"Bad": 0, "Present": 1, "Hot": 2}).astype(int)
new_df["Professional Scale"] = new_df["Professional Scale"].map({"Not Professional": 0, "Professional": 1}).astype(int)
new_df["Feminime Energy Scale"] = new_df["Feminime Energy Scale"].map({"Masculine": 0, "Ambivert": 1, "Feminime": 2}).astype(int)
new_df["Attitude Scale"] = new_df["Attitude Scale"].map({"Bad": 0, "Normal": 1, "Good": 2}).astype(int)
X = new_df[new_df.columns[:-1]].values
y = new_df[new_df.columns[-1]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [40]:
my_model = GirlTypeDecisionTree()
my_model.fit(X_train, y_train)
y_predict = my_model.predict(X_test)
sum(y_predict == y_test) / len(y_test)


/var/folders/48/_n6x3sr95hs3x66w52bj3krw0000gn/T/ipykernel_7164/3238888836.py:24: RuntimeWarning: invalid value encountered in scalar divide
  gini_impurity = 1 - sum([(count / N_child) ** 2 for count in girl_type_count.values()])


np.float64(0.984)

In [41]:
from sklearn.tree import DecisionTreeClassifier
sklearn_model = DecisionTreeClassifier()
sklearn_model.fit(X_train, y_train)
y_predict = my_model.predict(X_test)
sum(y_predict == y_test) / len(y_test)

np.float64(0.984)

In [42]:
from sklearn.ensemble import RandomForestClassifier
sklearn_model = RandomForestClassifier()
sklearn_model.fit(X_train, y_train)
y_predict = my_model.predict(X_test)
sum(y_predict == y_test) / len(y_test)

np.float64(0.984)